In [1]:
import os
import pickle

from GAMERNet.rnet.networks.networks import ReactionNetwork, is_closed_shell
from rdkit.Chem import rdDetermineBonds
from ase.visualize import view

with open('../scripts/ethylene/rxn_net.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)
print(rxn_net)

ReactionNetwork(47 intermediates, 25 closed-shell molecules, 86 reactions)



In [2]:
print(rxn_net.t_states)

[C3H6O+Co48<->C3H5O+H, C3H6O+Co48<->C3H5O+H, C3H6O+Co48<->C3H5O+H, C3H6O+Co48<->C3H5O+H, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->C3H4O+H, C3H5O+Co48<->C3H4O+H, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->C3H4O+H, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->C3H4O+H, C3H5O+Co48<->H+C3H4O, C3H5O+Co48<->H+C3H4O, Co48+C3H4O<->C3H3O+H, Co48+C3H4O<->H+C3H3O, Co48+C3H4O<->H+C3H3O, C3H4O+Co48<->C3H3O+H, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H4O+Co48<->H+C3H3O, C3H3O+Co48<->C3H2O+H, C3H3O+Co48<->H+C3H2O, C3H3O+Co48<->C3H2O+H, C3H3O+Co4

In [3]:
import random

for ts in rxn_net.t_states:
    # Provide a random energy for each transition state between 0 and 1.6 eV
    ts.energy = random.random() * 1.6

In [4]:
rxn_net.t_states[10].energy

0.16450629769115788

Look for elementary steps involving a specific intermediate

In [3]:
rxn_net.search_inter_by_elements({'C':1, 'H':0, 'O':2})

(102101(CO2),)

In [7]:
rxn_net.search_ts_1(["102101"])

Co48+C3H4O2<->C2H2O+CH2O

In [4]:
rxn_net.write_dotgraph2(".", 'rxn_net_propylene.png')

In [ ]:
t_state_test = rxn_net.t_states[10]
print(t_state_test)

In [ ]:
for i in t_state_test.components:
    for j in i:
        print(j.code)

In [ ]:
t_state_test.full_label()

In [ ]:
print(rxn_net)

In [ ]:
print(rxn_net.closed_shells)

cs_mols = []
for mol in rxn_net.closed_shells:
    cs_mols.append(mol.molecule)

view(cs_mols)

In [ ]:
# visualize all the intermediates with ase view

from ase.visualize import view

intermediates = []

for key in rxn_net.intermediates.keys():
    intermediates.append(rxn_net.intermediates[key].closed_shell)

# count how many intermediates are closed shell
none = 0
for i in intermediates:
    if i == None:
        none += 1

print(none)

Visualize reaction network

In [ ]:
rxn_net.draw_graph()

In [ ]:
rxn_net.write_dotgraph('rxn_net.png')

In [ ]:
rxn_net.search_inter_by_elements({'C':2, 'H':2, 'O':0})

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def check_shell(mol):
    # Calculate the total valence electrons
    total_valence_electrons = sum([atom.GetAtomicNum() - atom.GetFormalCharge() for atom in mol.GetAtoms()])

    # Calculate the total electrons needed for closed-shell (using octet rule for simplicity)
    total_closed_shell_electrons = sum([(8 - atom.GetTotalValence()) for atom in mol.GetAtoms()])

    # Check if there are any unpaired electrons
    unpaired_electrons = total_valence_electrons - total_closed_shell_electrons

    if unpaired_electrons == 0:
        return "closed-shell"
    else:
        return "open-shell"

# Create an RDKit molecule object for ethylene
ethylene = Chem.MolFromSmiles('C=C')
print(check_shell(ethylene))  # Should return "closed-shell"

# Create an RDKit molecule object for a methyl radical
methyl_radical = Chem.MolFromSmiles('[CH3]')
print(check_shell(methyl_radical))  # Should return "open-shell"


In [ ]:
from rdkit import Chem

def check_shell(mol):
    total_valence_electrons = 0
    required_electrons_for_closed_shell = 0
    
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        total_valence_electrons += atomic_num  # Sum of atomic numbers gives an estimate of the valence electrons
        required_electrons_for_closed_shell += 8  # Assuming octet rule for simplicity

    # Calculate unpaired electrons
    unpaired_electrons = required_electrons_for_closed_shell - total_valence_electrons
    
    # Estimate multiplicity (M = 2S + 1, where S is the total electron spin)
    multiplicity = unpaired_electrons + 1
    
    if multiplicity == 1:
        return "closed-shell"
    else:
        return "open-shell"

# Example usage
ethylene = Chem.MolFromSmiles('C=C')
print(check_shell(ethylene))  # Should be "closed-shell"

methyl_radical = Chem.MolFromSmiles('[CH3]')
print(check_shell(methyl_radical))  # Should be "open-shell"


# Check ASE atoms closed-shell nature

In [ ]:
def is_closed_shell(molecule: Chem.Mol):
    """
    Check if a molecule is closed-shell or not.
    """
    # symbols = molecule.get_chemical_symbols()
    # coords = molecule.get_positions()
    # xyz = '\n'.join(f'{symbol} {x} {y} {z}' for symbol, (x, y, z) in zip(symbols, coords))
    # xyz = "{}\n\n{}".format(len(molecule), xyz)
    # rdkit_mol = Chem.MolFromXYZBlock(xyz)
    # conn_mol = Chem.Mol(rdkit_mol)
    # rdDetermineBonds.DetermineConnectivity(conn_mol)
    Chem.SanitizeMol(molecule, Chem.SANITIZE_FINDRADICALS  ^ Chem.SANITIZE_SETHYBRIDIZATION)
    unpaired_electrons = 0
    for atom in molecule.GetAtoms():
        unpaired_electrons += atom.GetNumRadicalElectrons()
        
    if unpaired_electrons == 0:
        return True
    else:
        return False
    
tests = ["ethylene", "ethane", "ethyne", "ethanol", "Oxygen"]
smiles = ["C=C", "CC", "C#C", "CCO", "O", "[CH3]"]
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    print(f"{smile}: {is_closed_shell(mol)}")

Plot graphviz with edge as function of energy

In [13]:
import networkx as nx
import re
import random
graph = rxn_net.gen_graph2(".")
# try layout kamada_kawai_layout
pos = nx.kamada_kawai_layout(graph)
nx.set_node_attributes(graph, pos, 'pos')
plot = nx.drawing.nx_pydot.to_pydot(graph)
# text in the node is the node attribute formula
for node in plot.get_nodes():
    # all numbers in the node label are subscripted
    formula = node.get_attributes()['formula']
    closed_shell = node.get_attributes()['closed_shell']
    # print(formula, closed_shell)
    for num in re.findall(r'\d+', formula):
        SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
        formula = formula.replace(num, num.translate(SUB))
    figure = node.get_attributes()['fig']
    node.set_fontname("Arial")
    # Add figure as html-like label without table borders
    node.set_label(f"""<
    <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="0">
    <TR>
    <TD><IMG SRC="{figure}"/></TD>
    </TR>
    <TR>
    <TD>{formula}</TD>
    </TR>
    </TABLE>>""")
    node.set_style("filled")
    node.set_fillcolor("wheat")
    # set node shape as function of closed_shell attribute
    node.set_shape("ellipse")          
    node.set_width("1.5")
    node.set_height("1.5")
    node.set_fixedsize("true")
# make graph undirected
for edge in plot.get_edges():
    edge.set_dir("none")
    edge.set_arrowhead("none")
    edge.set_arrowtail("none")
    edge.set_arrowsize("1.0")
    edge.set_penwidth("0.5")
    edge.set_color("azure4")
    edge.set_fontname("Arial")
    # # add reaction energy as edge label (random number between 0 and 1.6, only first two decimals)
    # edge.set_label(f"""<
    # <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="0">
    # <TR>
    # <TD>{random.random()*1.6:.2f}</TD>
    # </TR>
    # </TABLE>>""")
    # edge.set_fontsize("10")
    # edge.set_fontcolor("azure4")
    energy = random.random()*1.6
    # define edge color as function of energy (continuous color scale)
    # if energy < 0.4:
    #     edge.set_color("darkgreen")
    # elif energy < 0.8:
    #     edge.set_color("green")
    # elif energy < 1.2:
    #     edge.set_color("yellow")
    # elif energy < 1.6:
    #     edge.set_color("red")
    # else:
    #     edge.set_color("darkred")

# Define plot in order to have it more square in shape

  
plot.set_nodesep(0.3)
plot.set_rankdir('LR')
plot.set_margin(0.2)
plot.set_pad(0.2)
plot.write_png("RXN.png")

/scratch/ipykernel_4016581/638345095.py:8: DeprecationWarning: nx.nx_pydot.to_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  plot = nx.drawing.nx_pydot.to_pydot(graph)
